# Processing video
In this colab notebook, we will show how to make dataset from video.

Enjoy!

# Preparations
Before start, make sure that you choose
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**


## Connect google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
folder_path = '/content/drive/MyDrive/dancegan/bruno_mars/'
target = '/content/drive/MyDrive/dancegan/nuras/keys/'
colab_openpose_video_path = '/content/drive/MyDrive/dancegan/bruno_mars/'
folder_path_json = '/content/drive/MyDrive/dancegan/bruno_mars/keys/'
folder_path_image = '/content/drive/MyDrive/dancegan/bruno_mars/frames/'

result = '/content/drive/MyDrive/dancegan/nuras/avesmooth/train_label/'
result_video = '/content/drive/MyDrive/dancegan/nuras/'

posenorm_results = '/content/drive/MyDrive/dancegan/bruno_mars/posenorm/'
avesmooth_results = '/content/drive/MyDrive/dancegan/bruno_mars/avesmooth/'

In [3]:
# folder_path = '/content/drive/MyDrive/dancegan/nuras/'
# target = '/content/drive/MyDrive/dancegan/nuras/keys/'
# colab_openpose_video_path = '/content/drive/MyDrive/dancegan/bruno_mars/'
# folder_path_json = '/content/drive/MyDrive/dancegan/nuras/keys/'
# folder_path_image = '/content/drive/MyDrive/dancegan/nuras/frames/'
# train_results = '/content/drive/MyDrive/dancegan/nuras/train/'
# posenorm_results = '/content/drive/MyDrive/dancegan/bruno_mars/posenorm/'
# avesmooth_results = '/content/drive/MyDrive/dancegan/bruno_mars/avesmooth'

## Set up environment

In [2]:
%%capture
!pip install dominate
!git clone https://github.com/OrionChocoPie/EverybodyDanceNow.git

import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

## Processing video

In [3]:
import os
from os.path import exists, join, basename, splitext

import re 

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

!mkdir '{folder_path_json}'
!mkdir '{folder_path_image}'
files = os.listdir(folder_path)
files.reverse()
for filename in files:
  if filename.endswith('.mp4') and not filename.endswith('-openpose.mp4'):
    print(filename)
    colab_video_path = folder_path + filename
    print(colab_video_path)
    colab_openpose_video_path = colab_video_path.replace('.mp4', '') + '-openpose.mp4'
    print(colab_openpose_video_path)
    if not exists(colab_openpose_video_path): 
      !ffmpeg -i '{colab_video_path}' -qscale:v 2 '{folder_path_image}/'output_%03d.jpg 
      files = os.listdir(folder_path_image)
      
      sorted_frames=[]
      for x in sorted_nicely(files):
          sorted_frames.append(x)
      '''
      count = 0
      step = 3
      
      for filename in sorted_frames:
        print(filename)
        if count% step !=0:
          !rm '{folder_path_image}''{filename}'
        if count % step == 0:
          print(count)
        count = count + 1
        #every third frame
      '''
      !cd openpose && ./build/examples/openpose/openpose.bin --hand --face --image_dir '{folder_path_image}' \
       --display 0 --render_pose  0\
       --write_json '{folder_path_json}' # -vframes 6000 
      

print('done!')

NameError: ignored

## Rename files

In [ ]:
import glob
import os
import re
def get_trailing_number(s):
	m = re.search(r'\d+$', s)
	return int(m.group()) if m else None
  
# Function to rename multiple files 
def main(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.json")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		#print(number_trailing)
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")#+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+'.json')
		count = count + 1

def main_2(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.jpg")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))#+".jpg") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".jpg") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".jpg") 
		count = count + 1
  
  
# Driver Code 
import sys

#print(sys.argv)
if sys.argv[2] == 'json':
	main(sys.argv[1], "output_",6) 
if sys.argv[2] == 'jpg':
	main_2(sys.argv[1], "output_",6) 

In [ ]:
%%capture

main(folder_path_json, "output_", 6, False)

main_2(folder_path_image, "output_", 6, False)

In [ ]:
!ls '{folder_path_json}' | wc -l

# Making dataset

In [ ]:
# !python /content/EverybodyDanceNow/data_prep/graph_train.py \
# --keypoints_dir '{folder_path_json}' \
# --frames_dir '{folder_path_image}' \
# --save_dir '{train_results}' \
#  --spread 0 5000 1 \
#  --facetexts

In [ ]:
!python /content/EverybodyDanceNow/data_prep/graph_avesmooth.py \
--keypoints_dir '{folder_path_json}' \
--frames_dir '{folder_path_image}' \
--save_dir '{avesmooth_results}' \
--spread 150 5000 1 \
--facetexts

In [ ]:
!ls '{path_images}' | wc -l

In [ ]:
!python /content/EverybodyDanceNow/data_prep/graph_posenorm.py \
--target_keypoints '{target}' \
--source_keypoints '{folder_path_json}' \
--source_shape 1080 1920 3 \
--target_shape 1080 1920 3 \
--source_frames '{folder_path_image}' \
--results '{result}' \
--calculate_scale_translation \
--target_spread 1 200 \
--source_spread 150 200 \
--format json \
--filestart "output_" \
--x_scaling 2 \
--y_scaling -5000 \
# --facetexts \
# --calculate_scale_translation  \
#filestart is the name of the file stub


In [ ]:
!rm '{result_video}'ffmpeg.mp4
!ffmpeg -framerate 30 -pattern_type glob -i '{result}''*.png' \
  -c:v libx264 -pix_fmt yuv420p '{result_video}'ffmpeg.mp4